In [1]:
import pandas as pd
from bert_score import BERTScorer

from src.config import substitution_rating_file, preference_rating_file
from src.config import preference_rating_scores

import warnings
warnings.filterwarnings('ignore')

In [2]:
# predictions
df = pd.read_csv(preference_rating_file)
# ground truth
df2 = pd.read_csv(substitution_rating_file)[['user', 'item_1']].drop_duplicates().reset_index(drop=True)
df2 = df2[['user', 'item_1']]
df2.columns = ['user', 'gt_item']

gt = df2.groupby('user')['gt_item'].apply(list).to_dict()
df['gt'] = df['user'].map(gt)

In [11]:
import time
start = time.time()
scorer = BERTScorer(lang="en", rescale_with_baseline=True)
end = time.time()
print(end - start)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


13.720400094985962


In [12]:
def get_scores(single_cands, multi_refs):
    P_mul, R_mul, F_mul = scorer.score([single_cands], [multi_refs])
    return P_mul.item(), R_mul.item(), F_mul.item()

def scores(line):
    return get_scores(line['choice'].lower(), line['gt_item'].lower())

In [3]:
d = pd.merge(df2[['user', 'gt_item']], df[['user', 'qn', 'choice',]], how='outer', on=['user'])

In [4]:
d

,user,gt_item,qn,choice
0,1,Bottle Water - Water,1,Jasmine Rice - Rice
1,1,Bottle Water - Water,1,Avocado
2,1,Bottle Water - Water,1,Donut Shop - Donut
3,1,Bottle Water - Water,1,Sour patches - Sour patchss
4,1,Bottle Water - Water,1,Apple
...,...,...,...,...
17495,48,Snapple green tea - Green Tea,5,kellogs - nutrigrain blueberry breakfast bar
17496,48,Snapple green tea - Green Tea,5,eggland's best - cage free vegetarian fed hens...
17497,48,Snapple green tea - Green Tea,5,subway - 6 inch turkey breast and ham on wheat
17498,48,Snapple green tea - Green Tea,5,kellogg's pop tarts - hot fudge sundae


In [15]:
import time
start = time.time()
d['P'], d['R'], d['F1'] = zip(*d.apply(scores, axis=1))
end = time.time()
print(end - start)

1997.0440566539764


In [16]:
cols = ['user', 'qn', 'gt_item', 'choice', 'P', 'R', 'F1']
d = d[cols]
# d = d[cols].rename(columns={'R':'BERTScore', 'F1':'BERTScore-F1'})
d.to_csv(preference_rating_scores['BERTScores'], index=False)

In [17]:
d.head(2)

,user,qn,gt_item,choice,P,R,F1
0,1,1,Bottle Water - Water,Jasmine Rice - Rice,0.288776,0.288825,0.289956
1,1,1,Bottle Water - Water,Avocado,0.107143,-0.078615,0.014121
